In [ ]:
import requests
import boto3
from botocore.exceptions import NoCredentialsError
import os

s3_bucket = 'rearc-data-quest-s3'
s3_prefix = 'bls-data/'
bls_url = 'https://download.bls.gov/pub/time.series/pr/pr.data.0.Current'
file_name = bls_url.split("/")[-1]
tmp_file_path = f'/tmp/{file_name}'

headers = {
    "User-Agent": "Ruchita Patil (patilruchita0792@gmail.com)"
}

def download_file(url, output_path):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {output_path}")
        return True
    else:
        print(f"Failed to download: {response.status_code}")
        return False

def upload_to_s3(local_path, bucket, s3_key):
    s3 = boto3.client('s3')
    try:
        s3.upload_file(local_path, bucket, s3_key)
        print(f"Uploaded to S3: s3://{bucket}/{s3_key}")
    except NoCredentialsError:
        print("AWS credentials not found.")
    except Exception as e:
        print("Upload error:", str(e))

def lambda_handler(event, context):
    if download_file(bls_url, tmp_file_path):
        upload_to_s3(tmp_file_path, s3_bucket, s3_prefix + file_name)
        return {
            'statusCode': 200,
            'body': f'Successfully uploaded {file_name} to s3://{s3_bucket}/{s3_prefix + file_name}'
        }
    else:
        return {
            'statusCode': 500,
            'body': 'Failed to download the file.'
        }
